In [1]:
!pip install sox
!pip install soundfile
!pip install torch
!pip install transformers
!pip install gradio
!pip install pyctcdecode
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install torchaudio
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 855.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sox: filename=sox-1.5.0-py3-none-any.whl size=40038 sha256=d768e8233ae4993f5823b737f17b0b2bebff8ae341a16fc4b716dc0fa6715cb7
  Stored in directory: /root/.cache/pip/wheels/74/e7/7b/8033be3ec5e4994595d01269fc9657c8fd83a0dcbf8536666a
Successfully built sox
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB

In [2]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor,Wav2Vec2ProcessorWithLM
import gradio as gr
import sox
import subprocess
import difflib
from random import sample

hindi_words = ["नमस्ते", "रानी", "भरत", "धन्यवाद", "स्वागत", "शुभकामनाएँ", "आशीर्वाद"]
def read_file_and_process(wav_file):
    filename = wav_file.split('.')[0]
    filename_16k = filename + "16k.wav"
    resampler(wav_file, filename_16k)
    speech, _ = sf.read(filename_16k)
    inputs = processor(speech, sampling_rate=16_000, return_tensors="pt", padding=True)
    return inputs


def resampler(input_file_path, output_file_path):
    command = (
    f"ffmpeg -hide_banner -loglevel panic -i {input_file_path} -ar 16000 -ac 1 -bits_per_raw_sample 16 -vn "
    f"{output_file_path}" )
    subprocess.call(command, shell=True)


def parse_transcription_with_lm(logits):
    result = processor_with_LM.batch_decode(logits.cpu().numpy())
    text = result.text
    transcription = text[0].replace('<s>','')
    return transcription

def parse_transcription(logits):
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

def highlight_diff(reference, transcribed):
    differ = difflib.Differ()
    diff= list(differ.compare(reference, transcribed))
    highlighted_text = ''
    for item in diff:
        if item[0] == '':
             highlighted_text += item[2]
        elif item[0] == '-':
             highlighted_text += f'<span style="color:green"> {item[2]} </span>'
        elif item[0] == '+':
             highlighted_text += f'<span style="color:red"> {item[2]} </span>'
    return highlighted_text

def play_audio(wav_file):
    command = f"ffplay -nodisp -autoexit {wav_file}"
    subprocess.call(command, shell=True)

def parse(wav_file, applyLM, reference_text):
    input_values = read_file_and_process(wav_file)
    with torch.no_grad():
         logits= model(**input_values).logits
    if applyLM:
         transcribed_text = parse_transcription_with_lm(logits)
    else:
         transcribed_text = parse_transcription(logits)
    comparison_result = highlight_diff(reference_text, transcribed_text)
    return transcribed_text, comparison_result

model_id = "Harveenchadha/vakyansh-wav2vec2-hindi-him-4200"
processor = Wav2Vec2Processor.from_pretrained(model_id)
processor_with_LM = Wav2Vec2ProcessorWithLM.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)


input= gr.Audio(sources="microphone", type="filepath")
reference_text = gr.Dropdown(label="रेफरन्स टेक्स्ट", choices=hindi_words)
transcribed_text = gr.HTML(label= "ट्रान्सक्रिप्शन")
comparison_result = gr.HTML(label= "कमप्यारिजन परिणाम:")
chkbox =gr. Checkbox(label="कमप्येअर करे", value=False)
iface= gr. Interface(fn=parse,
                     inputs=[input, chkbox, reference_text],
                     outputs=[transcribed_text, comparison_result])
iface.launch()


    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

kenLM.arpa:   0%|          | 0.00/2.01G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at Harveenchadha/vakyansh-wav2vec2-hindi-him-4200 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Harveenchadha/vakyansh-wav2vec2-hindi-him-4200 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4b944f94e5b48ad449.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
